# Portfolio Optimization - Case Study

## Business Overview
**Client**: Investment Management Firm  
**Objective**: Build optimized portfolios with superior risk-adjusted returns vs benchmark  
**Investment Horizon**: Medium to long-term (2+ years)

**Comparison Framework**: Equal-Weight vs sharpe-Optimized vs Benchmark (QQQ)

**Key Constraints**: 
- No short selling allowed
- No leverage permitted  
- Monthly rebalancing frequency

## Expected Deliverables
- Executive KPI dashboard with key metrics (CAGR, sharpe, MDD, Calmar)
- Risk-return visualization with efficient frontier
- Stress test analysis across market regimes
- Reproducible results with documented assumptions

# Configuration, Global Parameters, Logging & Reproducibility Setups

In [12]:
# CONFIGURATION & SETUP

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime, date
import warnings
import logging
import os
from itertools import combinations
from scipy.optimize import minimize
from scipy import stats

# Configure display and warnings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.options.display.float_format = '{:,.6f}'.format
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
%matplotlib inline


In [13]:
# GLOBAL PARAMETERS - EDIT THESE FOR DIFFERENT SCENARIOS

# Get the directory of the current notebook and navigate to the data folder
try:
    # Use __file__ when the notebook is run as a script (e.g., in a Binder or Colab environment)
    NOTEBOOK_DIR = os.path.dirname(os.path.abspath(__file__))
    DATA_FOLDER = os.path.join(NOTEBOOK_DIR, '..', 'data')
except NameError:
    # Fallback for local Jupyter environment
    NOTEBOOK_DIR = os.path.dirname(os.getcwd())
    DATA_FOLDER = os.path.join(NOTEBOOK_DIR, 'data')


# Stock universe (add more tickers as needed)
STOCK_UNIVERSE = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'TSLA', 'NFLX', 'AMD', 'CRM']
N_STOCKS_AUTO = 4  # Number of stocks to select automatically
START_DATE = '2020-01-01'
END_DATE = '2024-12-31'
MODE = 'snapshot'  # 'snapshot' or 'live'

# Risk Management
MAX_WEIGHT = 0.40  # Maximum allocation per asset (40%)
REBALANCE_FREQUENCY = 'M'  # Monthly rebalancing
COST_PER_SIDE = 0.001  # 10 bps transaction cost

# Benchmark & Analysis
BENCHMARK = 'QQQ'  # NASDAQ-100 ETF
TRAIN_END_DATE = '2023-12-31'  # Train/test split
MIN_DAYS = 500  # Minimum days required for analysis

# Monte Carlo & Reproducibility  
N_SCENARIOS = 20000  # Number of portfolio scenarios
RNG_SEED = 42  # For reproducible results
PORTFOLIO_VALUE = 10000  # Initial portfolio value ($10k)

# Walk-Forward Analysis Parameters
TRAIN_WINDOW = 252  # Training window in days (1 year)
TEST_WINDOW = 63   # Test window in days (3 months)
RISK_FREE_RATE = 0.02  # Risk-free rate for Sharpe calculation

In [14]:
# Set random seeds for reproducibility
np.random.seed(RNG_SEED)
rng = np.random.default_rng(RNG_SEED)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Utility functions for reproducibility
def _rng(seed=None):
    """Return a reproducible NumPy Generator."""
    return np.random.default_rng(RNG_SEED if seed is None else seed)

def _annualize_return(mean_daily):
    return mean_daily * 252.0

def _annualize_vol(std_daily):
    return std_daily * np.sqrt(252.0)

def _sharpe(mean, std, rf=0.0, eps=1e-12):
    return (mean - rf) / max(std, eps)

def _max_drawdown(nav: pd.Series) -> float:
    roll_max = nav.cummax()
    dd = nav / roll_max - 1.0
    return float(dd.min()) if len(dd) else 0.0

def _normalize_weights(w: np.ndarray, eps=1e-12):
    s = np.sum(w)
    if s <= eps:
        return np.zeros_like(w)
    return w / s

def _to_series_weights(w: np.ndarray, tickers):
    return pd.Series(w, index=list(tickers), dtype=float)

# Log key parameters
logger.info("=== PORTFOLIO OPTIMIZATION ENHANCED PARAMETERS ===")
logger.info(f"Mode: {MODE}")
logger.info(f"Stock Universe: {STOCK_UNIVERSE}")
logger.info(f"Auto-selection count: {N_STOCKS_AUTO}")
logger.info(f"Date Range: {START_DATE} to {END_DATE}")
logger.info(f"Benchmark: {BENCHMARK}")
logger.info(f"Max Weight per Asset: {MAX_WEIGHT*100}%")
logger.info(f"Transaction Cost: {COST_PER_SIDE*10000} bps per side")
logger.info(f"Monte Carlo Scenarios: {N_SCENARIOS:,}")
logger.info(f"Random Seed: {RNG_SEED}")
logger.info(f"Portfolio Value: ${PORTFOLIO_VALUE:,}")

print("✅ Configuration loaded successfully")
print(f"📊 Analysis Mode: {MODE.upper()}")
print(f"📈 Stock Universe: {len(STOCK_UNIVERSE)} stocks")
print(f"📅 Period: {START_DATE} to {END_DATE}")
print(f"🎯 Benchmark: {BENCHMARK}")

2025-08-22 17:05:39,039 - INFO - === PORTFOLIO OPTIMIZATION ENHANCED PARAMETERS ===
2025-08-22 17:05:39,040 - INFO - Mode: snapshot
2025-08-22 17:05:39,041 - INFO - Stock Universe: ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'TSLA', 'NFLX', 'AMD', 'CRM']
2025-08-22 17:05:39,042 - INFO - Auto-selection count: 4
2025-08-22 17:05:39,042 - INFO - Date Range: 2020-01-01 to 2024-12-31
2025-08-22 17:05:39,043 - INFO - Benchmark: QQQ
2025-08-22 17:05:39,044 - INFO - Max Weight per Asset: 40.0%
2025-08-22 17:05:39,044 - INFO - Transaction Cost: 10.0 bps per side
2025-08-22 17:05:39,045 - INFO - Monte Carlo Scenarios: 20,000
2025-08-22 17:05:39,045 - INFO - Random Seed: 42
2025-08-22 17:05:39,046 - INFO - Portfolio Value: $10,000


✅ Configuration loaded successfully
📊 Analysis Mode: SNAPSHOT
📈 Stock Universe: 10 stocks
📅 Period: 2020-01-01 to 2024-12-31
🎯 Benchmark: QQQ


## DATA INGESTION & VALIDATION FUNCTIONS

In [ ]:
# DATA INGESTION & VALIDATION FUNCTIONS

In [15]:
def load_prices_from_csv(tickers, start_date, end_date, data_folder='./', strict=True):
    """
    Load stock prices from CSV files with comprehensive validation
    
    Parameters:
    -----------
    tickers : list
        List of stock ticker symbols
    start_date : str
        Start date in 'YYYY-MM-DD' format
    end_date : str  
        End date in 'YYYY-MM-DD' format
    data_folder : str
        Path to folder containing CSV files
    strict : bool
        If True, raise errors for data issues; if False, issue warnings
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with aligned stock prices (Close Price only)
    """
    
    logger.info(f"Loading price data for {len(tickers)} tickers...")
    
    stock_data = {}
    failed_tickers = []
    
    for ticker in tickers:
        try:
            # Construct file path
            file_path = os.path.join(data_folder, f"{ticker}.csv")
            
            if not os.path.exists(file_path):
                logger.warning(f"File not found: {file_path}")
                failed_tickers.append(ticker)
                continue
            
            # Load CSV with date parsing
            df = pd.read_csv(file_path, parse_dates=['Date'], index_col='Date')
            
            # Filter date range
            df = df.loc[start_date:end_date]
            
            if df.empty:
                logger.warning(f"No data for {ticker} in specified date range")
                failed_tickers.append(ticker)
                continue
            
            # Store only Close Price
            stock_data[ticker] = df['Close Price']
            logger.info(f"✅ {ticker}: {len(df)} records loaded")
            
        except Exception as e:
            logger.error(f"Error loading {ticker}: {str(e)}")
            failed_tickers.append(ticker)
    
    if not stock_data:
        raise ValueError("No valid stock data loaded!")
    
    # Create aligned DataFrame
    stock_adj_close = pd.DataFrame(stock_data)
    
    # Data validation
    stock_adj_close = validate_stock_data(stock_adj_close, tickers, failed_tickers, strict)
    
    logger.info(f"📊 Final dataset: {len(stock_adj_close.columns)} stocks, {len(stock_adj_close)} days")
    return stock_adj_close

def validate_stock_data(df, original_tickers, failed_tickers, strict=True):
    """
    Comprehensive data validation and cleaning
    """
    logger.info("Performing data validation...")
    
    # Report failed tickers
    if failed_tickers:
        logger.warning(f"Failed to load: {failed_tickers}")
    
    # Check minimum data requirements
    if len(df) < MIN_DAYS:
        message = f"Dataset has only {len(df)} days (minimum: {MIN_DAYS})"
        if strict:
            raise ValueError(message)
        else:
            logger.warning(message)
    
    # Handle missing values
    missing_data = df.isnull().sum()
    if missing_data.any():
        logger.warning("Missing data detected:")
        for ticker, missing_count in missing_data[missing_data > 0].items():
            logger.warning(f"  {ticker}: {missing_count} missing values")
        
        # Forward fill missing values
        df = df.fillna(method='ffill').fillna(method='bfill')
        logger.info("Missing values filled using forward/backward fill")
    
    # Check for duplicated dates
    if df.index.duplicated().any():
        logger.warning("Duplicate dates found - removing duplicates")
        df = df[~df.index.duplicated(keep='first')]
    
    # Remove stocks with insufficient data
    min_valid_ratio = 0.95  # Require 95% valid data
    for ticker in df.columns.copy():
        valid_ratio = df[ticker].notna().sum() / len(df)
        if valid_ratio < min_valid_ratio:
            logger.warning(f"Removing {ticker}: only {valid_ratio:.1%} valid data")
            df = df.drop(columns=[ticker])
    
    # Check for extreme outliers and constant series
    returns = df.pct_change().dropna()
    for ticker in returns.columns.copy():
        # Check for constant prices (no variation)
        if returns[ticker].std() < 1e-6:
            logger.warning(f"Removing {ticker}: constant price series")
            df = df.drop(columns=[ticker])
            continue
        
        # Check for extreme movements (>50% daily change)
        extreme_moves = (abs(returns[ticker]) > 0.5).sum()
        if extreme_moves > 0:
            logger.warning(f"{ticker}: {extreme_moves} extreme daily moves (>50%)")
    
    if df.empty:
        raise ValueError("No valid stocks remaining after validation!")
    
    logger.info(f"✅ Validation complete: {len(df.columns)} stocks, {len(df)} days")
    return df

def compute_returns(prices_df: pd.DataFrame) -> pd.DataFrame:
    """Compute daily returns from prices (pct_change), drop first NaNs."""
    returns = prices_df.pct_change().dropna(how='all')
    return returns.replace([np.inf, -np.inf], np.nan).dropna(axis=0, how='all')

## ENHANCED KPI CALCULATION FUNCTIONS

In [ ]:
# ENHANCED KPI CALCULATION FUNCTIONS

In [16]:
def calculate_comprehensive_kpis(returns: pd.Series, benchmark_returns: pd.Series = None, rf: float = RISK_FREE_RATE) -> dict:
    """
    Calculate comprehensive KPIs including advanced risk metrics
    
    Parameters:
    -----------
    returns : pd.Series
        Portfolio returns series
    benchmark_returns : pd.Series, optional
        Benchmark returns for alpha/beta calculation
    rf : float
        Risk-free rate (annualized)
    
    Returns:
    --------
    dict
        Dictionary with comprehensive KPIs
    """
    if len(returns) == 0:
        return {"ann_return": 0.0, "ann_vol": 0.0, "sharpe": 0.0, "max_dd": 0.0,
                "sortino": 0.0, "var_95": 0.0, "cvar_95": 0.0, "alpha": 0.0, "beta": 0.0, "info_ratio": 0.0}
    
    # Basic metrics
    mu_d = float(returns.mean())
    sd_d = float(returns.std(ddof=0))
    ann_r = _annualize_return(mu_d)
    ann_v = _annualize_vol(sd_d)
    sr = _sharpe(ann_r, ann_v, rf)
    
    # Maximum Drawdown
    nav = (1 + returns).cumprod()
    mdd = _max_drawdown(nav)
    
    # Sortino Ratio (downside deviation)
    downside_returns = returns[returns < 0]
    downside_std = float(downside_returns.std(ddof=0)) if len(downside_returns) > 0 else 0.0
    sortino = _sharpe(ann_r, _annualize_vol(downside_std), rf) if downside_std > 0 else 0.0
    
    # Value at Risk (VaR) and Conditional VaR (CVaR) at 95% confidence
    var_95 = float(np.percentile(returns, 5)) * np.sqrt(252)  # Annualized
    cvar_95 = float(returns[returns <= np.percentile(returns, 5)].mean()) * np.sqrt(252)  # Annualized
    
    # Alpha, Beta, and Information Ratio vs benchmark
    alpha, beta, info_ratio = 0.0, 0.0, 0.0
    if benchmark_returns is not None and len(benchmark_returns) > 0:
        # Align returns
        aligned_idx = returns.index.intersection(benchmark_returns.index)
        if len(aligned_idx) > 10:  # Need sufficient data
            port_aligned = returns.reindex(aligned_idx).fillna(0.0)
            bench_aligned = benchmark_returns.reindex(aligned_idx).fillna(0.0)
            
            # Beta calculation
            if bench_aligned.std() > 0:
                beta = float(np.cov(port_aligned, bench_aligned)[0, 1] / np.var(bench_aligned))
            
            # Alpha calculation (annualized)
            bench_ann_return = _annualize_return(bench_aligned.mean())
            alpha = ann_r - (rf + beta * (bench_ann_return - rf))
            
            # Information Ratio
            active_returns = port_aligned - bench_aligned
            tracking_error = _annualize_vol(active_returns.std(ddof=0))
            if tracking_error > 0:
                info_ratio = _annualize_return(active_returns.mean()) / tracking_error
    
    return {
        "ann_return": ann_r,
        "ann_vol": ann_v,
        "sharpe": sr,
        "max_dd": mdd,
        "sortino": sortino,
        "var_95": var_95,
        "cvar_95": cvar_95,
        "alpha": alpha,
        "beta": beta,
        "info_ratio": info_ratio,
        "mu_d": mu_d,
        "vol_d": sd_d
    }

def compute_kpis(returns: pd.Series, rf: float = 0.0) -> dict:
    """Compute basic KPIs from a return series (backward compatibility)."""
    if len(returns) == 0:
        return {"ann_return": 0.0, "ann_vol": 0.0, "sharpe": 0.0, "max_dd": 0.0}
    mu_d = float(returns.mean())
    sd_d = float(returns.std(ddof=0))
    ann_r = _annualize_return(mu_d)
    ann_v = _annualize_vol(sd_d)
    sr = _sharpe(ann_r, ann_v, rf)
    nav = (1 + returns).cumprod()
    mdd = _max_drawdown(nav)
    return {"ann_return": ann_r, "ann_vol": ann_v, "sharpe": sr, "max_dd": mdd}

def kpis_vs_benchmark(port_ret: pd.Series, bench_ret: pd.Series) -> dict:
    """Compute correlation, tracking error, and active return vs benchmark."""
    idx = port_ret.index.intersection(bench_ret.index)
    pr = port_ret.reindex(idx).fillna(0.0)
    br = bench_ret.reindex(idx).fillna(0.0)
    corr = float(np.corrcoef(pr, br)[0,1]) if pr.std() > 0 and br.std() > 0 else 0.0
    active = pr - br
    te = float(active.std(ddof=0) * np.sqrt(252.0))
    ar = float((_annualize_return(pr.mean()) - _annualize_return(br.mean())))
    return {"corr": corr, "tracking_error": te, "active_return": ar}

## PORTFOLIO OPTIMIZATION FUNCTIONS

In [ ]:
# PORTFOLIO OPTIMIZATION FUNCTIONS

In [17]:
def average_offdiag_correlation(returns_df: pd.DataFrame) -> float:
    """Average off-diagonal correlation among assets."""
    if returns_df.shape[1] <= 1:
        return 0.0
    corr = returns_df.corr().values
    n = corr.shape[0]
    return float((corr.sum() - np.trace(corr)) / (n * (n - 1)))

def portfolio_stats_from_weights(returns_df: pd.DataFrame, weights: np.ndarray, rf: float = 0.0) -> dict:
    """Compute mean, vol, sharpe given asset returns and weights."""
    w = np.asarray(weights).reshape(-1)
    mu = returns_df.mean().values
    cov = returns_df.cov().values
    pmu_d = float(np.dot(w, mu))
    pvol_d = float(np.sqrt(max(np.dot(w, cov @ w), 0.0)))
    ann_r = _annualize_return(pmu_d)
    ann_v = _annualize_vol(pvol_d)
    sr = _sharpe(ann_r, ann_v, rf)
    return {"ann_return": ann_r, "ann_vol": ann_v, "sharpe": sr, "mu_d": pmu_d, "vol_d": pvol_d}

def optimize_weights(
    returns_df: pd.DataFrame,
    objective: str = 'max_sharpe',
    bounds: tuple = (0.0, MAX_WEIGHT),
    sum_to: float = 1.0,
    method: str = 'scipy',
    turnover_penalty: float = 0.0,
    prev_weights: np.ndarray = None,
    risk_free: float = RISK_FREE_RATE,
    seed: int = None,
) -> dict:
    """
    Optimize portfolio weights under constraints with enhanced objectives
    
    Parameters:
    -----------
    returns_df : pd.DataFrame
        Asset returns DataFrame
    objective : str
        'max_sharpe', 'min_var', or 'risk_parity'
    bounds : tuple
        (lower_bound, upper_bound) for weights
    sum_to : float
        Target sum of weights (usually 1.0)
    method : str
        'scipy' or 'mc' (Monte Carlo)
    turnover_penalty : float
        Penalty for turnover (L1 norm of weight changes)
    prev_weights : np.ndarray
        Previous weights for turnover calculation
    risk_free : float
        Risk-free rate
    seed : int
        Random seed
    
    Returns:
    --------
    dict
        Dictionary with optimized weights, KPIs, and method used
    """
    assets = list(returns_df.columns)
    n = len(assets)
    lb, ub = bounds
    lb = max(0.0, float(lb))
    ub = float(ub)
    if ub <= 0:
        raise ValueError("Upper bound must be > 0")

    mu = returns_df.mean().values
    cov = returns_df.cov().values
    x0 = np.full(n, sum_to / n, dtype=float)
    x0 = np.clip(x0, lb, ub)
    x0 = _normalize_weights(x0) * sum_to
    prev_w = np.zeros(n) if prev_weights is None else np.asarray(prev_weights).reshape(-1)

    def obj_sharpe(w):
        w = np.asarray(w)
        pmu_d = np.dot(w, mu)
        pvar = max(np.dot(w, cov @ w), 0.0)
        pstd_d = np.sqrt(pvar)
        ann_r = _annualize_return(pmu_d)
        ann_v = _annualize_vol(pstd_d)
        sharpe = _sharpe(ann_r, ann_v, risk_free)
        pen = turnover_penalty * np.sum(np.abs(w - prev_w)) if turnover_penalty > 0 else 0.0
        return -sharpe + pen

    def obj_minvar(w):
        w = np.asarray(w)
        pen = turnover_penalty * np.sum(np.abs(w - prev_w)) if turnover_penalty > 0 else 0.0
        return max(np.dot(w, cov @ w), 0.0) + pen

    def obj_risk_parity(w):
        """Risk Parity objective: minimize sum of squared deviations from equal risk contribution"""
        w = np.asarray(w)
        covw = cov @ w
        rc = w * covw  # risk contributions using variance
        total_var = max(float(w @ covw), 1e-12)
        target = total_var / n
        pen = turnover_penalty * np.sum(np.abs(w - prev_w)) if turnover_penalty > 0 else 0.0
        return float(np.sum((rc - target) ** 2)) + pen

    def project_bounds(w):
        w = np.clip(w, lb, ub)
        if w.sum() <= 0:
            return np.full_like(w, sum_to / len(w))
        return w / w.sum() * sum_to

    # Try SciPy optimization first
    if method == 'scipy':
        try:
            cons = (
                {'type': 'eq', 'fun': lambda w: np.sum(w) - sum_to},
            )
            bnds = tuple((lb, ub) for _ in range(n))
            
            if objective == 'max_sharpe':
                fun = obj_sharpe
            elif objective == 'min_var':
                fun = obj_minvar
            elif objective == 'risk_parity':
                fun = obj_risk_parity
            else:
                raise ValueError(f"Unknown objective: {objective}")
            
            res = minimize(fun, x0, method='SLSQP', bounds=bnds, constraints=cons, 
                         options={'maxiter': 200, 'ftol': 1e-9})
            
            if res.success:
                w_opt = project_bounds(res.x)
                stats = portfolio_stats_from_weights(returns_df, w_opt, rf=risk_free)
                return {
                    'weights': _to_series_weights(w_opt, assets),
                    'kpis': stats,
                    'method_used': 'scipy',
                }
        except Exception as e:
            logger.warning(f"SciPy optimization failed: {e}, falling back to Monte Carlo")
    
    # Monte Carlo fallback
    logger.info("Using Monte Carlo optimization")
    rng_local = _rng(seed)
    W = rng_local.dirichlet(alpha=np.ones(n), size=N_SCENARIOS)
    mask = (W <= ub + 1e-12).all(axis=1)
    Wf = W[mask]
    
    if Wf.shape[0] < max(1000, n * 200):
        Wc = np.clip(W, lb, ub)
        Wc = (Wc / Wc.sum(axis=1, keepdims=True)) * sum_to
        Wf = Wc
    else:
        Wf = (Wf / Wf.sum(axis=1, keepdims=True)) * sum_to

    if objective == 'max_sharpe':
        pmu_d = Wf @ mu
        pvar = np.einsum('ij,jk,ik->i', Wf, cov, Wf)
        pstd_d = np.sqrt(np.maximum(pvar, 0.0))
        ann_r = _annualize_return(pmu_d)
        ann_v = _annualize_vol(pstd_d)
        sharpe = (ann_r - risk_free) / np.maximum(ann_v, 1e-12)
        if turnover_penalty > 0:
            pen = turnover_penalty * np.sum(np.abs(Wf - prev_w.reshape(1, -1)), axis=1)
            sharpe = sharpe - pen
        idx = int(np.argmax(sharpe))
        w_opt = Wf[idx]
    elif objective == 'min_var':
        pvar = np.einsum('ij,jk,ik->i', Wf, cov, Wf)
        if turnover_penalty > 0:
            pen = turnover_penalty * np.sum(np.abs(Wf - prev_w.reshape(1, -1)), axis=1)
            score = pvar + pen
            idx = int(np.argmin(score))
        else:
            idx = int(np.argmin(pvar))
        w_opt = Wf[idx]
    elif objective == 'risk_parity':
        covW = Wf @ cov  # (m, n)
        rc = Wf * covW   # (m, n)
        total_var = np.sum(rc, axis=1)
        target = (total_var / n).reshape(-1, 1)
        score = np.sum((rc - target) ** 2, axis=1)
        if turnover_penalty > 0:
            score = score + turnover_penalty * np.sum(np.abs(Wf - prev_w.reshape(1, -1)), axis=1)
        idx = int(np.argmin(score))
        w_opt = Wf[idx]
    else:
        raise ValueError(f"Unknown objective: {objective}")

    w_opt = project_bounds(w_opt)
    stats = portfolio_stats_from_weights(returns_df, w_opt, rf=risk_free)
    return {
        'weights': _to_series_weights(w_opt, assets),
        'kpis': stats,
        'method_used': 'mc',
    }

## STOCK SELECTION FUNCTIONS

In [ ]:
# STOCK SELECTION FUNCTIONS

In [19]:
def select_optimal_stocks(
    prices_df: pd.DataFrame,
    N_stocks: int,
    lookback_days: int,
    method: str = 'greedy',
    seed: int = None,
) -> dict:
    """
    Select N_stocks from the universe using advanced selection methodology
    
    Parameters:
    -----------
    prices_df : pd.DataFrame
        Price data for all assets
    N_stocks : int
        Number of stocks to select
    lookback_days : int
        Number of days to look back for analysis
    method : str
        'exhaustive' for small universes, 'greedy' for larger ones
    seed : int
        Random seed for reproducibility
    
    Returns:
    --------
    dict
        Dictionary with selected tickers, analysis DataFrame, and best weights
    """
    assert N_stocks >= 1, "N_stocks must be >= 1"
    tickers = list(prices_df.columns)
    end_idx = prices_df.index[-1]
    start_idx = prices_df.index[max(0, len(prices_df) - lookback_days)]
    px_lb = prices_df.loc[start_idx:end_idx]
    rets_lb = compute_returns(px_lb)
    combos_rows = []

    if len(tickers) <= 10 and method == 'exhaustive':
        logger.info(f"Using exhaustive search for {len(tickers)} assets")
        best = None
        for combo in combinations(tickers, N_stocks):
            sub = rets_lb[list(combo)].dropna()
            if sub.shape[0] < 2:
                continue
            opt = optimize_weights(sub, objective='max_sharpe', bounds=(0.0, MAX_WEIGHT), 
                                 sum_to=1.0, method='scipy', seed=seed)
            k = opt['kpis']
            avg_corr = average_offdiag_correlation(sub)
            score = k['sharpe']
            combos_rows.append({
                'combo': combo,
                'sharpe': k['sharpe'],
                'ann_return': k['ann_return'],
                'ann_vol': k['ann_vol'],
                'avg_corr': avg_corr,
                'score': score,
                'weights': opt['weights'],
            })
            if best is None or score > best['score']:
                best = combos_rows[-1]
        combos_df = pd.DataFrame(combos_rows)
        sel = list(best['combo']) if best else tickers[:N_stocks]
        best_weights = best['weights'] if best else pd.Series(np.full(N_stocks, 1.0 / N_stocks), index=sel)
        return {'selected_tickers': sel, 'combos_df': combos_df, 'best_weights': best_weights}

    # Greedy selection for larger universes
    logger.info(f"Using greedy search for {len(tickers)} assets")
    selected = []
    remaining = set(tickers)
    best_weights = None
    
    while len(selected) < N_stocks and remaining:
        best_local = None
        for cand in list(remaining):
            trial = selected + [cand]
            sub = rets_lb[trial].dropna()
            if sub.shape[0] < 2:
                continue
            opt = optimize_weights(sub, objective='max_sharpe', bounds=(0.0, MAX_WEIGHT), 
                                 sum_to=1.0, method='scipy', seed=seed)
            avg_corr = average_offdiag_correlation(sub)
            score = opt['kpis']['sharpe'] * (1.0 - avg_corr)  # Penalize high correlation
            row = {
                'combo': tuple(trial),
                'sharpe': opt['kpis']['sharpe'],
                'ann_return': opt['kpis']['ann_return'],
                'ann_vol': opt['kpis']['ann_vol'],
                'avg_corr': avg_corr,
                'score': score,
                'weights': opt['weights'],
            }
            if best_local is None or score > best_local['score']:
                best_local = row
        
        if best_local is None:
            break
        
        selected = list(best_local['combo'])
        best_weights = best_local['weights']
        remaining = set(tickers) - set(selected)
        combos_rows.append(best_local)
    
    combos_df = pd.DataFrame(combos_rows)
    if len(selected) > N_stocks:
        selected = selected[:N_stocks]
        best_weights = best_weights[selected]
        best_weights = best_weights / best_weights.sum()
    
    logger.info(f"Selected {len(selected)} stocks: {selected}")
    return {'selected_tickers': selected, 'combos_df': combos_df, 'best_weights': best_weights}

## SIMULATION & REBALANCING FUNCTIONS

In [ ]:
# SIMULATION & REBALANCING FUNCTIONS

In [21]:
def build_weights_schedule(start_date: pd.Timestamp, end_date: pd.Timestamp, weights: pd.Series, freq: str = 'M') -> pd.DataFrame:
    """Create a schedule with constant weights at a given frequency between dates (inclusive)."""
    idx = pd.date_range(start=start_date, end=end_date, freq=freq)
    if len(idx) == 0:
        idx = pd.DatetimeIndex([start_date])
    ws = pd.DataFrame(index=idx, columns=weights.index, dtype=float)
    for d in ws.index:
        ws.loc[d] = weights.values
    return ws

def simulate_rebalance(
    prices_df: pd.DataFrame,
    weights_schedule: pd.DataFrame,
    rebalance_freq: str = 'M',
    cost_per_side: float = COST_PER_SIDE,
    slippage: float = 0.0,
) -> dict:
    """
    Simulate NAV with rebalancing at close, applying costs and tracking turnover
    
    Parameters:
    -----------
    prices_df : pd.DataFrame
        Price data for assets
    weights_schedule : pd.DataFrame
        Rebalancing schedule (index: dates, columns: tickers)
    rebalance_freq : str
        Rebalancing frequency
    cost_per_side : float
        Transaction cost per side
    slippage : float
        Additional slippage cost
    
    Returns:
    --------
    dict
        Dictionary with NAV series, turnover, costs, and KPIs
    """
    px = prices_df.copy()
    px = px[sorted(px.columns)]
    rets = compute_returns(px)
    ws = weights_schedule.copy().reindex(rets.index).dropna(how='all').fillna(0.0)
    all_cols = sorted(set(px.columns) | set(ws.columns))
    rets = rets.reindex(columns=all_cols).fillna(0.0)
    ws = ws.reindex(columns=all_cols).fillna(0.0)

    nav = 1.0
    current_weights = np.zeros(len(all_cols), dtype=float)
    nav_series = pd.Series(index=rets.index, dtype=float)
    turnover_series = pd.Series(0.0, index=rets.index)
    cum_cost_value = 0.0
    cum_cost_events = 0

    for t in rets.index:
        r_t = rets.loc[t].values
        day_ret = float(np.nansum(current_weights * r_t))
        nav *= (1.0 + day_ret)

        if t in ws.index and not ws.loc[t].isna().all():
            target = ws.loc[t].values.astype(float)
            w_eod_unnorm = current_weights * (1.0 + r_t)
            s = w_eod_unnorm.sum()
            w_eod = w_eod_unnorm / s if s > 0 else current_weights.copy()
            delta = target - w_eod
            turnover = float(np.sum(np.abs(delta)))
            cost = (cost_per_side + slippage) * turnover * nav
            nav -= cost
            cum_cost_value += float(cost)
            cum_cost_events += 1 if turnover > 0 else 0
            turnover_series.loc[t] = turnover
            current_weights = target.copy()

        nav_series.loc[t] = nav

    returns_net = nav_series.pct_change().fillna(0.0)
    kpis_net = compute_kpis(returns_net)
    return {
        'nav': nav_series,
        'turnover': turnover_series[turnover_series > 0],
        'cum_cost': float(cum_cost_value),
        'cum_cost_events': float(cum_cost_events),
        'kpis_net': kpis_net,
    }

## WALK-FORWARD ANALYSIS FUNCTIONS

In [ ]:
# WALK-FORWARD ANALYSIS FUNCTIONS

In [22]:
def walk_forward_evaluate(
    prices_df: pd.DataFrame,
    train_window: int = TRAIN_WINDOW,
    test_window: int = TEST_WINDOW,
    rebalance_freq: str = REBALANCE_FREQUENCY,
    params: dict = None,
) -> dict:
    """
    Walk-forward validation pipeline with comprehensive analysis
    
    Parameters:
    -----------
    prices_df : pd.DataFrame
        Price data for all assets
    train_window : int
        Training window size in days
    test_window : int
        Test window size in days
    rebalance_freq : str
        Rebalancing frequency
    params : dict
        Parameters for optimization
    
    Returns:
    --------
    dict
        Dictionary with fold results, aggregated KPIs, and OOS NAV
    """
    if params is None:
        params = {
            'N_stocks': N_STOCKS_AUTO,
            'selection_method': 'exhaustive' if len(prices_df.columns) <= 10 else 'greedy',
            'objective': 'max_sharpe',
            'opt_method': 'scipy',
            'turnover_penalty': 0.0,
            'cost_per_side': COST_PER_SIDE,
            'max_weight': MAX_WEIGHT,
            'seed': RNG_SEED,
        }
    
    px = prices_df.copy()
    dates = px.index
    folds = []
    all_nav = []
    i = train_window
    prev_w = None
    rng_seed = params.get('seed', RNG_SEED)
    
    logger.info(f"Starting walk-forward analysis: train={train_window}, test={test_window}")
    
    while i + test_window <= len(dates):
        train_start = dates[i - train_window]
        train_end = dates[i - 1]
        test_start = dates[i]
        test_end = dates[i + test_window - 1]
        
        logger.info(f"Fold: Train {train_start.date()} to {train_end.date()}, Test {test_start.date()} to {test_end.date()}")
        
        px_tr = px.loc[train_start:train_end]

        # Stock selection
        sel = select_optimal_stocks(
            px_tr,
            N_stocks=params.get('N_stocks', N_STOCKS_AUTO),
            lookback_days=train_window,
            method=params.get('selection_method', 'exhaustive' if px_tr.shape[1] <= 10 else 'greedy'),
            seed=rng_seed,
        )
        tickers = sel['selected_tickers']
        
        # Weight optimization
        rets_tr = compute_returns(px_tr[tickers]).dropna()
        opt = optimize_weights(
            rets_tr,
            objective=params.get('objective', 'max_sharpe'),
            bounds=(0.0, params.get('max_weight', MAX_WEIGHT)),
            sum_to=1.0,
            method=params.get('opt_method', 'scipy'),
            turnover_penalty=params.get('turnover_penalty', 0.0),
            prev_weights=prev_w if prev_w is not None else None,
            seed=rng_seed,
        )
        w = opt['weights']
        prev_w = w.reindex(tickers).fillna(0.0).values

        # Out-of-sample simulation
        ws = build_weights_schedule(test_start, test_end, w, freq=rebalance_freq)
        sim = simulate_rebalance(
            px[tickers].loc[dates[0]:test_end], 
            ws, 
            rebalance_freq=rebalance_freq, 
            cost_per_side=params.get('cost_per_side', COST_PER_SIDE)
        )
        nav = sim['nav'].loc[test_start:test_end]
        all_nav.append(nav)
        
        # Calculate KPIs
        kpis_is = opt['kpis']
        kpis_oos = compute_kpis(nav.pct_change().fillna(0.0))
        
        folds.append({
            'train_start': train_start, 'train_end': train_end,
            'test_start': test_start, 'test_end': test_end,
            'tickers': tickers, 'weights': w,
            'kpis_is': kpis_is, 'kpis_oos': kpis_oos,
        })
        i += test_window

    # Aggregate results
    nav_oos = pd.concat(all_nav).sort_index() if all_nav else pd.Series(dtype=float)
    agg_kpis = compute_kpis(nav_oos.pct_change().fillna(0.0)) if len(nav_oos) else {k: 0.0 for k in ('ann_return','ann_vol','sharpe','max_dd')}
    
    logger.info(f"Walk-forward complete: {len(folds)} folds, OOS Sharpe: {agg_kpis.get('sharpe', 0):.3f}")
    
    return {'folds': folds, 'agg_kpis_oos': agg_kpis, 'nav_oos': nav_oos}

## REPORTING & SUMMARY FUNCTIONS

In [ ]:
# REPORTING & SUMMARY FUNCTIONS

In [23]:
def generate_exec_summary(kpis_dict: dict) -> str:
    """
    Generate executive summary with key insights
    
    Parameters:
    -----------
    kpis_dict : dict
        Dictionary with 'is', 'oos', and 'costs' keys
    
    Returns:
    --------
    str
        Markdown formatted executive summary
    """
    is_k = kpis_dict.get('is', {})
    oos_k = kpis_dict.get('oos', {})
    costs = kpis_dict.get('costs', {})
    
    lines = [
        "## 📊 Executive Summary",
        "",
        f"**Performance Overview:**",
        f"- In-sample Sharpe Ratio: {is_k.get('sharpe', 0):.2f} | Out-of-sample Sharpe: {oos_k.get('sharpe', 0):.2f}",
        f"- Out-of-sample Annual Return: {oos_k.get('ann_return', 0)*100:.2f}% | Volatility: {oos_k.get('ann_vol', 0)*100:.2f}%",
        f"- Maximum Drawdown: {oos_k.get('max_dd', 0)*100:.2f}%",
        "",
        f"**Risk Management:**",
        f"- Rebalancing events: {costs.get('events', 0):.0f}",
        f"- Transaction cost per side: {costs.get('cost_per_side', COST_PER_SIDE)*10000:.1f} bps",
        "",
        f"**Recommendations:**",
        f"- {'✅ Strong' if oos_k.get('sharpe', 0) > 1.0 else '⚠️ Moderate' if oos_k.get('sharpe', 0) > 0.5 else '❌ Weak'} risk-adjusted performance",
        f"- {'✅ Acceptable' if abs(oos_k.get('max_dd', 0)) < 0.2 else '⚠️ High'} drawdown levels",
        f"- Monitor tracking error and consider adjusting weight constraints if needed",
    ]
    return "\n".join(lines)

def load_benchmark_data(prices_df: pd.DataFrame, benchmark_ticker: str = BENCHMARK) -> pd.Series:
    """
    Load benchmark data or create synthetic benchmark
    
    Parameters:
    -----------
    prices_df : pd.DataFrame
        Universe price data
    benchmark_ticker : str
        Benchmark ticker symbol
    
    Returns:
    --------
    pd.Series
        Benchmark price series
    """
    # Try to load benchmark from data folder
    benchmark_path = os.path.join(DATA_FOLDER, f"{benchmark_ticker}.csv")
    
    if os.path.exists(benchmark_path):
        try:
            benchmark_df = load_prices_from_csv([benchmark_ticker], START_DATE, END_DATE, DATA_FOLDER, strict=False)
            if not benchmark_df.empty and benchmark_ticker in benchmark_df.columns:
                benchmark_series = benchmark_df[benchmark_ticker].reindex(prices_df.index).fillna(method='ffill')
                logger.info(f"✅ Loaded {benchmark_ticker} benchmark data")
                return benchmark_series
        except Exception as e:
            logger.warning(f"Failed to load {benchmark_ticker}: {e}")
    
    # Create synthetic equal-weight benchmark
    logger.info(f"Creating synthetic equal-weight benchmark from universe")
    returns = compute_returns(prices_df)
    if returns.empty:
        return pd.Series(index=prices_df.index, dtype=float)
    
    w = np.full(len(prices_df.columns), 1.0 / max(1, len(prices_df.columns)))
    ew_ret = pd.Series(np.nansum(returns.values * w.reshape(1, -1), axis=1), index=returns.index)
    ew_nav = (1 + ew_ret).cumprod()
    return ew_nav / ew_nav.iloc[0] * 100  # Normalize to 100

## ENHANCED VISUALIZATION FUNCTIONS

In [ ]:
# ENHANCED VISUALIZATION FUNCTIONS

In [24]:
def create_performance_dashboard(portfolio_nav, benchmark_nav, portfolio_kpis, benchmark_kpis, weights_series):
    """
    Create comprehensive performance dashboard with multiple charts
    
    Parameters:
    -----------
    portfolio_nav : pd.Series
        Portfolio NAV series
    benchmark_nav : pd.Series
        Benchmark NAV series
    portfolio_kpis : dict
        Portfolio KPIs
    benchmark_kpis : dict
        Benchmark KPIs
    weights_series : pd.Series
        Final portfolio weights
    
    Returns:
    --------
    plotly.graph_objects.Figure
        Interactive dashboard figure
    """
    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Portfolio vs Benchmark Performance',
            'Portfolio Allocation',
            'Rolling Drawdown',
            'Performance Metrics Comparison'
        ),
        specs=[[{"secondary_y": False}, {"type": "pie"}],
               [{"secondary_y": False}, {"type": "bar"}]],
        vertical_spacing=0.12,
        horizontal_spacing=0.1
    )
    
    # 1. Performance comparison
    fig.add_trace(
        go.Scatter(
            x=portfolio_nav.index,
            y=portfolio_nav.values,
            name='Portfolio',
            line=dict(color='#1f77b4', width=2)
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=benchmark_nav.index,
            y=benchmark_nav.values,
            name='Benchmark',
            line=dict(color='#ff7f0e', width=2, dash='dash')
        ),
        row=1, col=1
    )
    
    # 2. Portfolio allocation pie chart
    fig.add_trace(
        go.Pie(
            labels=weights_series.index,
            values=weights_series.values,
            name="Portfolio Weights",
            textinfo='label+percent',
            textposition='auto'
        ),
        row=1, col=2
    )
    
    # 3. Rolling drawdown
    portfolio_dd = (portfolio_nav / portfolio_nav.cummax() - 1) * 100
    benchmark_dd = (benchmark_nav / benchmark_nav.cummax() - 1) * 100
    
    fig.add_trace(
        go.Scatter(
            x=portfolio_dd.index,
            y=portfolio_dd.values,
            name='Portfolio DD',
            fill='tonexty',
            line=dict(color='red', width=1)
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=benchmark_dd.index,
            y=benchmark_dd.values,
            name='Benchmark DD',
            line=dict(color='orange', width=1, dash='dash')
        ),
        row=2, col=1
    )
    
    # 4. Performance metrics comparison
    metrics = ['ann_return', 'ann_vol', 'sharpe', 'max_dd']
    portfolio_values = [portfolio_kpis.get(m, 0) * (100 if m in ['ann_return', 'ann_vol', 'max_dd'] else 1) for m in metrics]
    benchmark_values = [benchmark_kpis.get(m, 0) * (100 if m in ['ann_return', 'ann_vol', 'max_dd'] else 1) for m in metrics]
    
    fig.add_trace(
        go.Bar(
            x=['Annual Return (%)', 'Volatility (%)', 'Sharpe Ratio', 'Max DD (%)'],
            y=portfolio_values,
            name='Portfolio',
            marker_color='#1f77b4'
        ),
        row=2, col=2
    )
    
    fig.add_trace(
        go.Bar(
            x=['Annual Return (%)', 'Volatility (%)', 'Sharpe Ratio', 'Max DD (%)'],
            y=benchmark_values,
            name='Benchmark',
            marker_color='#ff7f0e'
        ),
        row=2, col=2
    )
    
    # Update layout
    fig.update_layout(
        title={
            'text': '📊 Portfolio Optimization Dashboard',
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 20}
        },
        height=800,
        showlegend=True,
        template='plotly_white'
    )
    
    # Update axes labels
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_yaxes(title_text="NAV", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Drawdown (%)", row=2, col=1)
    fig.update_yaxes(title_text="Value", row=2, col=2)
    
    return fig

def create_risk_metrics_chart(comprehensive_kpis):
    """
    Create advanced risk metrics visualization
    
    Parameters:
    -----------
    comprehensive_kpis : dict
        Dictionary with comprehensive KPIs including VaR, CVaR, etc.
    
    Returns:
    --------
    plotly.graph_objects.Figure
        Risk metrics chart
    """
    # Risk metrics data
    risk_metrics = {
        'Sharpe Ratio': comprehensive_kpis.get('sharpe', 0),
        'Sortino Ratio': comprehensive_kpis.get('sortino', 0),
        'Information Ratio': comprehensive_kpis.get('info_ratio', 0),
        'VaR (95%)': abs(comprehensive_kpis.get('var_95', 0)) * 100,
        'CVaR (95%)': abs(comprehensive_kpis.get('cvar_95', 0)) * 100,
        'Max Drawdown': abs(comprehensive_kpis.get('max_dd', 0)) * 100
    }
    
    # Create radar chart for risk metrics
    fig = go.Figure()
    
    # Normalize values for radar chart (except ratios)
    normalized_values = []
    for metric, value in risk_metrics.items():
        if 'Ratio' in metric:
            normalized_values.append(max(0, min(value, 3)))  # Cap ratios at 3
        else:
            normalized_values.append(value)
    
    fig.add_trace(go.Scatterpolar(
        r=normalized_values,
        theta=list(risk_metrics.keys()),
        fill='toself',
        name='Risk Metrics',
        line_color='rgb(1,90,120)',
        fillcolor='rgba(1,90,120,0.2)'
    ))
    
    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, max(normalized_values) * 1.1]
            )
        ),
        title={
            'text': '🎯 Advanced Risk Metrics Profile',
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 16}
        },
        template='plotly_white',
        height=500
    )
    
    return fig

def create_walk_forward_analysis_chart(walk_forward_results):
    """
    Create walk-forward analysis visualization
    
    Parameters:
    -----------
    walk_forward_results : dict
        Results from walk_forward_evaluate function
    
    Returns:
    --------
    plotly.graph_objects.Figure
        Walk-forward analysis chart
    """
    folds = walk_forward_results['folds']
    nav_oos = walk_forward_results['nav_oos']
    
    # Create subplots
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=(
            'Out-of-Sample NAV Evolution',
            'In-Sample vs Out-of-Sample Sharpe Ratios by Fold'
        ),
        vertical_spacing=0.15
    )
    
    # 1. OOS NAV evolution
    fig.add_trace(
        go.Scatter(
            x=nav_oos.index,
            y=nav_oos.values,
            name='OOS NAV',
            line=dict(color='#2E8B57', width=2)
        ),
        row=1, col=1
    )
    
    # 2. IS vs OOS Sharpe ratios
    fold_numbers = list(range(1, len(folds) + 1))
    is_sharpe = [fold['kpis_is']['sharpe'] for fold in folds]
    oos_sharpe = [fold['kpis_oos']['sharpe'] for fold in folds]
    
    fig.add_trace(
        go.Bar(
            x=fold_numbers,
            y=is_sharpe,
            name='In-Sample Sharpe',
            marker_color='#4CAF50',
            opacity=0.7
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Bar(
            x=fold_numbers,
            y=oos_sharpe,
            name='Out-of-Sample Sharpe',
            marker_color='#FF9800',
            opacity=0.7
        ),
        row=2, col=1
    )
    
    # Update layout
    fig.update_layout(
        title={
            'text': '🔄 Walk-Forward Analysis Results',
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 18}
        },
        height=700,
        showlegend=True,
        template='plotly_white'
    )
    
    # Update axes
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_yaxes(title_text="NAV", row=1, col=1)
    fig.update_xaxes(title_text="Fold Number", row=2, col=1)
    fig.update_yaxes(title_text="Sharpe Ratio", row=2, col=1)
    
    return fig

## MAIN EXECUTION PIPELINE

In [ ]:
# MAIN EXECUTION PIPELINE

### Step 1: Data Loading and Validation

In [25]:
# Step 1: Load and validate stock price data
print("🔄 Cloning repo and locating data...")

# Clone repo if needed
REPO_URL = "https://github.com/maxiveloso/python-portfolio-optimization.git"
REPO_DIR = "/content/python-portfolio-optimization"
DATA_FOLDER = os.path.join(REPO_DIR, "data")

if not os.path.exists(REPO_DIR):
    print("📥 Cloning GitHub repo...")
    !git clone -q {REPO_URL} {REPO_DIR}

# Confirm data files are present
files = os.listdir(DATA_FOLDER)
tickers_found = [f.replace(".csv", "") for f in files if f.endswith(".csv")]
print(f"📂 Found {len(tickers_found)} tickers in repo data folder: {', '.join(tickers_found[:10])}...")

# Load data using your existing function
try:
    stock_prices = load_prices_from_csv(
        tickers=STOCK_UNIVERSE,
        start_date=START_DATE,
        end_date=END_DATE,
        data_folder=DATA_FOLDER,
        strict=False
    )
    
    benchmark_prices = load_benchmark_data(stock_prices, BENCHMARK)
    
    print(f"✅ Data loaded successfully:")
    print(f"   📈 Stocks: {len(stock_prices.columns)} assets, {len(stock_prices)} days")
    print(f"   🎯 Benchmark: {len(benchmark_prices)} days")
    print(f"   📅 Date range: {stock_prices.index[0].date()} to {stock_prices.index[-1].date()}")
    
    # Display basic statistics
    returns = compute_returns(stock_prices)
    print(f"\n📊 Universe Statistics:")
    print(f"   Average daily return: {returns.mean().mean()*100:.3f}%")
    print(f"   Average daily volatility: {returns.std().mean()*100:.3f}%")
    print(f"   Average correlation: {average_offdiag_correlation(returns):.3f}")
    
except Exception as e:
    print(f"❌ Error loading data: {e}")
    raise

2025-08-22 17:06:32,574 - INFO - Loading price data for 10 tickers...
2025-08-22 17:06:32,575 - WARNING - File not found: /Users/maximvf/Library/CloudStorage/GoogleDrive-maxiveloso@gmail.com/Mi unidad/Learning/Business Intelligence Data Analysis (BIDA)/data/AAPL.csv
2025-08-22 17:06:32,576 - WARNING - File not found: /Users/maximvf/Library/CloudStorage/GoogleDrive-maxiveloso@gmail.com/Mi unidad/Learning/Business Intelligence Data Analysis (BIDA)/data/MSFT.csv
2025-08-22 17:06:32,577 - WARNING - File not found: /Users/maximvf/Library/CloudStorage/GoogleDrive-maxiveloso@gmail.com/Mi unidad/Learning/Business Intelligence Data Analysis (BIDA)/data/GOOGL.csv
2025-08-22 17:06:32,577 - WARNING - File not found: /Users/maximvf/Library/CloudStorage/GoogleDrive-maxiveloso@gmail.com/Mi unidad/Learning/Business Intelligence Data Analysis (BIDA)/data/AMZN.csv
2025-08-22 17:06:32,578 - WARNING - File not found: /Users/maximvf/Library/CloudStorage/GoogleDrive-maxiveloso@gmail.com/Mi unidad/Learning/B

🔄 Loading stock price data...
❌ Error loading data: No valid stock data loaded!


ValueError: No valid stock data loaded!

### Step 2: Stock Selection and Portfolio Optimization

In [13]:
# Step 2: Stock selection and portfolio optimization
print("🔄 Performing stock selection and optimization...")

# Stock selection
lookback_days = min(504, len(stock_prices))  # 2 years or available data
selection_method = 'exhaustive' if len(stock_prices.columns) <= 10 else 'greedy'

selection_results = select_optimal_stocks(
    prices_df=stock_prices,
    N_stocks=N_STOCKS_AUTO,
    lookback_days=lookback_days,
    method=selection_method,
    seed=RNG_SEED
)

selected_tickers = selection_results['selected_tickers']
print(f"✅ Selected {len(selected_tickers)} stocks: {selected_tickers}")

# Portfolio optimization with multiple objectives
selected_prices = stock_prices[selected_tickers]
selected_returns = compute_returns(selected_prices)

# Optimize for different objectives
objectives = ['max_sharpe', 'min_var', 'risk_parity']
optimization_results = {}

for objective in objectives:
    print(f"\n🎯 Optimizing for {objective}...")
    opt_result = optimize_weights(
        returns_df=selected_returns,
        objective=objective,
        bounds=(0.0, MAX_WEIGHT),
        sum_to=1.0,
        method='scipy',
        risk_free=RISK_FREE_RATE,
        seed=RNG_SEED
    )
    optimization_results[objective] = opt_result
    
    kpis = opt_result['kpis']
    print(f"   📊 {objective}: Sharpe={kpis['sharpe']:.3f}, Return={kpis['ann_return']*100:.2f}%, Vol={kpis['ann_vol']*100:.2f}%")

# Use max_sharpe as primary strategy
primary_weights = optimization_results['max_sharpe']['weights']
primary_kpis = optimization_results['max_sharpe']['kpis']

print(f"\n🏆 Primary Strategy (Max Sharpe):")
print(f"   Sharpe Ratio: {primary_kpis['sharpe']:.3f}")
print(f"   Annual Return: {primary_kpis['ann_return']*100:.2f}%")
print(f"   Annual Volatility: {primary_kpis['ann_vol']*100:.2f}%")
print(f"\n💼 Portfolio Weights:")
for ticker, weight in primary_weights.items():
    print(f"   {ticker}: {weight*100:.2f}%")

2025-08-21 19:02:00,670 - INFO - Using exhaustive search for 10 assets


🔄 Performing stock selection and optimization...
✅ Selected 4 stocks: ['NVDA', 'META', 'NFLX', 'CRM']

🎯 Optimizing for max_sharpe...
   📊 max_sharpe: Sharpe=1.182, Return=48.48%, Vol=39.31%

🎯 Optimizing for min_var...
   📊 min_var: Sharpe=0.759, Return=28.63%, Vol=35.11%

🎯 Optimizing for risk_parity...
   📊 risk_parity: Sharpe=1.041, Return=40.10%, Vol=36.61%

🏆 Primary Strategy (Max Sharpe):
   Sharpe Ratio: 1.182
   Annual Return: 48.48%
   Annual Volatility: 39.31%

💼 Portfolio Weights:
   NVDA: 40.00%
   META: 24.11%
   NFLX: 27.62%
   CRM: 8.28%


### Step 3: Comprehensive KPI Analysis

In [14]:
# Step 3: Calculate comprehensive KPIs
print("🔄 Calculating comprehensive KPIs...")

# Calculate portfolio returns
portfolio_returns = (selected_returns * primary_weights).sum(axis=1)
benchmark_returns = compute_returns(benchmark_prices.to_frame('benchmark'))['benchmark']

# Calculate comprehensive KPIs
comprehensive_kpis = calculate_comprehensive_kpis(
    returns=portfolio_returns,
    benchmark_returns=benchmark_returns,
    rf=RISK_FREE_RATE
)

benchmark_kpis = calculate_comprehensive_kpis(
    returns=benchmark_returns,
    rf=RISK_FREE_RATE
)

# Display comprehensive metrics
print(f"\n📊 Comprehensive Portfolio Analysis:")
print(f"   📈 Annual Return: {comprehensive_kpis['ann_return']*100:.2f}%")
print(f"   📉 Annual Volatility: {comprehensive_kpis['ann_vol']*100:.2f}%")
print(f"   ⚡ Sharpe Ratio: {comprehensive_kpis['sharpe']:.3f}")
print(f"   🎯 Sortino Ratio: {comprehensive_kpis['sortino']:.3f}")
print(f"   📊 Information Ratio: {comprehensive_kpis['info_ratio']:.3f}")
print(f"   🔻 Maximum Drawdown: {comprehensive_kpis['max_dd']*100:.2f}%")
print(f"   ⚠️ VaR (95%): {comprehensive_kpis['var_95']*100:.2f}%")
print(f"   🚨 CVaR (95%): {comprehensive_kpis['cvar_95']*100:.2f}%")
print(f"   🔢 Alpha: {comprehensive_kpis['alpha']*100:.2f}%")
print(f"   📐 Beta: {comprehensive_kpis['beta']:.3f}")

# Benchmark comparison
vs_benchmark = kpis_vs_benchmark(portfolio_returns, benchmark_returns)
print(f"\n🎯 vs {BENCHMARK} Benchmark:")
print(f"   📊 Correlation: {vs_benchmark['corr']:.3f}")
print(f"   📈 Active Return: {vs_benchmark['active_return']*100:.2f}%")
print(f"   📉 Tracking Error: {vs_benchmark['tracking_error']*100:.2f}%")

🔄 Calculating comprehensive KPIs...

📊 Comprehensive Portfolio Analysis:
   📈 Annual Return: 48.48%
   📉 Annual Volatility: 39.30%
   ⚡ Sharpe Ratio: 1.183
   🎯 Sortino Ratio: 1.648
   📊 Information Ratio: 1.363
   🔻 Maximum Drawdown: -62.58%
   ⚠️ VaR (95%): -60.50%
   🚨 CVaR (95%): -88.51%
   🔢 Alpha: 21.29%
   📐 Beta: 1.357

🎯 vs QQQ Benchmark:
   📊 Correlation: 0.884
   📈 Active Return: 27.91%
   📉 Tracking Error: 20.48%


### Step 4: Walk-Forward Analysis

In [15]:
# Step 4: Walk-forward validation
print("🔄 Performing walk-forward analysis...")

# Walk-forward parameters
wf_params = {
    'N_stocks': N_STOCKS_AUTO,
    'selection_method': selection_method,
    'objective': 'max_sharpe',
    'opt_method': 'scipy',
    'turnover_penalty': 0.0,
    'cost_per_side': COST_PER_SIDE,
    'max_weight': MAX_WEIGHT,
    'seed': RNG_SEED,
}

# Perform walk-forward analysis
walk_forward_results = walk_forward_evaluate(
    prices_df=stock_prices,
    train_window=TRAIN_WINDOW,
    test_window=TEST_WINDOW,
    rebalance_freq=REBALANCE_FREQUENCY,
    params=wf_params
)

# Display walk-forward results
oos_kpis = walk_forward_results['agg_kpis_oos']
n_folds = len(walk_forward_results['folds'])

print(f"\n🔄 Walk-Forward Analysis Results ({n_folds} folds):")
print(f"   📈 OOS Annual Return: {oos_kpis['ann_return']*100:.2f}%")
print(f"   📉 OOS Annual Volatility: {oos_kpis['ann_vol']*100:.2f}%")
print(f"   ⚡ OOS Sharpe Ratio: {oos_kpis['sharpe']:.3f}")
print(f"   🔻 OOS Maximum Drawdown: {oos_kpis['max_dd']*100:.2f}%")

# Calculate average IS vs OOS performance
avg_is_sharpe = np.mean([fold['kpis_is']['sharpe'] for fold in walk_forward_results['folds']])
avg_oos_sharpe = np.mean([fold['kpis_oos']['sharpe'] for fold in walk_forward_results['folds']])

print(f"\n📊 Average Performance Across Folds:")
print(f"   🎯 Average IS Sharpe: {avg_is_sharpe:.3f}")
print(f"   🎯 Average OOS Sharpe: {avg_oos_sharpe:.3f}")
print(f"   📉 Performance Decay: {((avg_is_sharpe - avg_oos_sharpe) / avg_is_sharpe * 100):.1f}%")

2025-08-21 19:02:37,951 - INFO - Starting walk-forward analysis: train=252, test=63
2025-08-21 19:02:37,952 - INFO - Fold: Train 2020-01-02 to 2020-12-30, Test 2020-12-31 to 2021-04-01
2025-08-21 19:02:37,957 - INFO - Using exhaustive search for 10 assets


🔄 Performing walk-forward analysis...


2025-08-21 19:02:38,358 - INFO - Fold: Train 2020-04-02 to 2021-04-01, Test 2021-04-05 to 2021-07-01
2025-08-21 19:02:38,360 - INFO - Using exhaustive search for 10 assets
2025-08-21 19:02:38,782 - INFO - Fold: Train 2020-07-02 to 2021-07-01, Test 2021-07-02 to 2021-09-30
2025-08-21 19:02:38,784 - INFO - Using exhaustive search for 10 assets
2025-08-21 19:02:39,188 - INFO - Fold: Train 2020-10-01 to 2021-09-30, Test 2021-10-01 to 2021-12-30
2025-08-21 19:02:39,190 - INFO - Using exhaustive search for 10 assets
2025-08-21 19:02:39,581 - INFO - Fold: Train 2020-12-31 to 2021-12-30, Test 2021-12-31 to 2022-03-31
2025-08-21 19:02:39,583 - INFO - Using exhaustive search for 10 assets
2025-08-21 19:02:39,990 - INFO - Fold: Train 2021-04-05 to 2022-03-31, Test 2022-04-01 to 2022-07-01
2025-08-21 19:02:39,991 - INFO - Using exhaustive search for 10 assets
2025-08-21 19:02:40,357 - INFO - Fold: Train 2021-07-02 to 2022-07-01, Test 2022-07-05 to 2022-09-30
2025-08-21 19:02:40,358 - INFO - Using 


🔄 Walk-Forward Analysis Results (15 folds):
   📈 OOS Annual Return: 9.20%
   📉 OOS Annual Volatility: 38.59%
   ⚡ OOS Sharpe Ratio: 0.238
   🔻 OOS Maximum Drawdown: -36.79%

📊 Average Performance Across Folds:
   🎯 Average IS Sharpe: 1.887
   🎯 Average OOS Sharpe: 0.488
   📉 Performance Decay: 74.1%


### Step 5: Portfolio Simulation with Rebalancing

In [16]:
# Step 5: Portfolio simulation with rebalancing
print("🔄 Simulating portfolio with rebalancing...")

# Create rebalancing schedule
simulation_start = stock_prices.index[0]
simulation_end = stock_prices.index[-1]

weights_schedule = build_weights_schedule(
    start_date=simulation_start,
    end_date=simulation_end,
    weights=primary_weights,
    freq=REBALANCE_FREQUENCY
)

# Run simulation
simulation_results = simulate_rebalance(
    prices_df=selected_prices,
    weights_schedule=weights_schedule,
    rebalance_freq=REBALANCE_FREQUENCY,
    cost_per_side=COST_PER_SIDE,
    slippage=0.0
)

portfolio_nav = simulation_results['nav']
turnover_events = simulation_results['turnover']
total_costs = simulation_results['cum_cost']
cost_events = simulation_results['cum_cost_events']

# Calculate final portfolio value
final_value = portfolio_nav.iloc[-1] * PORTFOLIO_VALUE
total_return = (portfolio_nav.iloc[-1] - 1) * 100

print(f"\n💰 Portfolio Simulation Results:")
print(f"   🏁 Final Portfolio Value: ${final_value:,.2f}")
print(f"   📈 Total Return: {total_return:.2f}%")
print(f"   🔄 Rebalancing Events: {cost_events:.0f}")
print(f"   💸 Total Transaction Costs: ${total_costs * PORTFOLIO_VALUE:.2f}")
print(f"   📊 Average Turnover per Event: {turnover_events.mean()*100:.2f}%")

# Benchmark comparison
benchmark_nav = (1 + benchmark_returns).cumprod()
benchmark_final_value = benchmark_nav.iloc[-1] * PORTFOLIO_VALUE
benchmark_total_return = (benchmark_nav.iloc[-1] - 1) * 100

print(f"\n🎯 vs {BENCHMARK} Benchmark:")
print(f"   📈 Portfolio Return: {total_return:.2f}%")
print(f"   📊 Benchmark Return: {benchmark_total_return:.2f}%")
print(f"   🏆 Excess Return: {(total_return - benchmark_total_return):.2f}%")

🔄 Simulating portfolio with rebalancing...

💰 Portfolio Simulation Results:
   🏁 Final Portfolio Value: $75,498.35
   📈 Total Return: 654.98%
   🔄 Rebalancing Events: 42
   💸 Total Transaction Costs: $23.85
   📊 Average Turnover per Event: 3.48%

🎯 vs QQQ Benchmark:
   📈 Portfolio Return: 654.98%
   📊 Benchmark Return: 136.51%
   🏆 Excess Return: 518.48%


### Step 6: Executive Summary Generation

In [17]:
# Step 6: Generate executive summary
print("🔄 Generating executive summary...")

# Prepare summary data
summary_data = {
    'is': comprehensive_kpis,
    'oos': oos_kpis,
    'costs': {
        'events': cost_events,
        'cost_per_side': COST_PER_SIDE
    }
}

# Generate executive summary
exec_summary = generate_exec_summary(summary_data)

print("\n" + "="*60)
print(exec_summary)
print("="*60)

🔄 Generating executive summary...

## 📊 Executive Summary

**Performance Overview:**
- In-sample Sharpe Ratio: 1.18 | Out-of-sample Sharpe: 0.24
- Out-of-sample Annual Return: 9.20% | Volatility: 38.59%
- Maximum Drawdown: -36.79%

**Risk Management:**
- Rebalancing events: 42
- Transaction cost per side: 10.0 bps

**Recommendations:**
- ❌ Weak risk-adjusted performance
- ⚠️ High drawdown levels
- Monitor tracking error and consider adjusting weight constraints if needed


### Step 7: Enhanced Visualizations

In [26]:
# Step 7: Create enhanced visualizations
print("🔄 Creating enhanced visualizations...")

# 1. Performance Dashboard
dashboard_fig = create_performance_dashboard(
    portfolio_nav=portfolio_nav,
    benchmark_nav=benchmark_nav,
    portfolio_kpis=comprehensive_kpis,
    benchmark_kpis=benchmark_kpis,
    weights_series=primary_weights
)

dashboard_fig.show()
try:
    dashboard_fig.write_html('/home/ubuntu/portfolio_dashboard.html')
except Exception as e:
    print("Could not write dashboard to /home/ubuntu/portfolio_dashboard.html — file not saved. (", e, ")")

# 2. Risk Metrics Chart
risk_fig = create_risk_metrics_chart(comprehensive_kpis)
risk_fig.show()
try:
    risk_fig.write_html('/home/ubuntu/risk_metrics.html')
except Exception as e:
    print("Could not write dashboard to /home/ubuntu/risk_metrics.html — file not saved. (", e, ")")

# 3. Walk-Forward Analysis Chart
wf_fig = create_walk_forward_analysis_chart(walk_forward_results)
wf_fig.show()
try:
    wf_fig.write_html('/home/ubuntu/walk_forward_analysis.html')
except Exception as e:
    print("Could not write dashboard to /home/ubuntu/walk_forward_analysis.html — file not saved. (", e, ")")


print("\n✅ Visualizations created and saved:")
print("   📊 portfolio_dashboard.html")
print("   🎯 risk_metrics.html")
print("   🔄 walk_forward_analysis.html")

🔄 Creating enhanced visualizations...


NameError: name 'portfolio_nav' is not defined

### Step 8: Export Results

In [ ]:
# Step 8: Export results to CSV files
print("🔄 Exporting results...")

# Export portfolio weights
primary_weights.to_csv('/home/ubuntu/final_portfolio_weights.csv', header=['Weight'])

# Export NAV series
nav_export = pd.DataFrame({
    'Date': portfolio_nav.index,
    'Portfolio_NAV': portfolio_nav.values,
    'Benchmark_NAV': benchmark_nav.reindex(portfolio_nav.index).fillna(method='ffill').values
})
nav_export.to_csv('/home/ubuntu/nav_series.csv', index=False)

# Export comprehensive KPIs
kpis_export = pd.DataFrame({
    'Metric': list(comprehensive_kpis.keys()),
    'Portfolio': list(comprehensive_kpis.values()),
    'Benchmark': [benchmark_kpis.get(k, np.nan) for k in comprehensive_kpis.keys()]
})
kpis_export.to_csv('/home/ubuntu/comprehensive_kpis.csv', index=False)

# Export walk-forward results
wf_export = []
for i, fold in enumerate(walk_forward_results['folds']):
    wf_export.append({
        'Fold': i + 1,
        'Train_Start': fold['train_start'],
        'Train_End': fold['train_end'],
        'Test_Start': fold['test_start'],
        'Test_End': fold['test_end'],
        'IS_Sharpe': fold['kpis_is']['sharpe'],
        'IS_Return': fold['kpis_is']['ann_return'],
        'IS_Vol': fold['kpis_is']['ann_vol'],
        'OOS_Sharpe': fold['kpis_oos']['sharpe'],
        'OOS_Return': fold['kpis_oos']['ann_return'],
        'OOS_Vol': fold['kpis_oos']['ann_vol'],
        'Selected_Tickers': '|'.join(fold['tickers'])
    })

wf_df = pd.DataFrame(wf_export)
wf_df.to_csv('/home/ubuntu/walk_forward_results.csv', index=False)

# Export executive summary
with open('/home/ubuntu/executive_summary.md', 'w') as f:
    f.write(exec_summary)

print("\n✅ Results exported successfully:")
print("   💼 final_portfolio_weights.csv")
print("   📈 nav_series.csv")
print("   📊 comprehensive_kpis.csv")
print("   🔄 walk_forward_results.csv")
print("   📝 executive_summary.md")

print("\n🎉 Portfolio optimization analysis complete!")
print(f"\n🏆 Final Results Summary:")
print(f"   Selected Assets: {', '.join(selected_tickers)}")
print(f"   Portfolio Sharpe Ratio: {comprehensive_kpis['sharpe']:.3f}")
print(f"   Annual Return: {comprehensive_kpis['ann_return']*100:.2f}%")
print(f"   Maximum Drawdown: {comprehensive_kpis['max_dd']*100:.2f}%")
print(f"   Walk-Forward OOS Sharpe: {oos_kpis['sharpe']:.3f}")